<a href="https://colab.research.google.com/github/ReynaldoHdz/Cosechadores-Multiagentes/blob/main/tractorPrimitivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install agentpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00


In [ ]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

ModuleNotFoundError: ignored

In [ ]:
class Terrain(ap.Agent):
    def setup(self):
        self.random = self.model.random
        mylist = [True, False]
        #self.obstacle = self.random.choice(mylist)
        #self.wheat = self.random.choice(mylist)
        self.obstacle = False
        self.wheat = True
    def trimWheat(self):
        self.wheat = False

class Harvester(ap.Agent):
    def setup(self):
        self.random = self.model.random
        self.grid_harvester = self.model.grid_harvester
        self.grid_terrain = self.model.grid_terrain
        self.pos = [(0, 0)]
        allTerrainPositions = self.grid_terrain.positions

        # list out keys and values separately
        key_list = list(allTerrainPositions.keys())
        val_list = list(allTerrainPositions.values())
        # print key with val 0,0
        #print(allTerrainPositions)
        position = val_list.index((0,0))
        self.currTerrain = key_list[position]
        #print(key_list[position])
        #print(self.currTerrain)
        self.wheatInTank = 0
        self.numValid_Neighbors = 3

    def ObtainValid_neighbors(self):
        neighbors = self.grid_terrain.neighbors(self.currTerrain, distance=1)
        #self.valid_neighbors = [pos for pos in neighbors if pos.wheat == True and pos.obstacle == False]
        allTerrainPositions = self.grid_terrain.positions

        # Filtrar vecinos válidos que estén al norte, sur, este u oeste

        self.valid_neighbors = [
            pos for pos in neighbors if (
                pos.wheat and not pos.obstacle and
                (allTerrainPositions[pos][0] == allTerrainPositions[currTerrain][0] and abs(allTerrainPositions[pos][1] - allTerrainPositions[currTerrain][1]) == 1) or
                (allTerrainPositions[pos][1] == allTerrainPositions[pos][1] and abs(allTerrainPositions[pos][0] - allTerrainPositions[currTerrain][0]) == 1)
            )
        ]
        self.numValid_Neighbors = len(self.valid_neighbors)


    def moveToNoObstacle(self):
        #print(valid_neighbors)
        if len(self.valid_neighbors) > 0:
            self.currTerrain.trimWheat()
            self.wheatInTank += 1
            #print(self.wheatInTank)
            randomNeighbor = self.random.choice(self.valid_neighbors)
            allTerrainPositions = self.grid_terrain.positions
            self.currTerrain = randomNeighbor
            self.pos = allTerrainPositions[randomNeighbor]
    def getnumValid_Neighbors(self):
        return self.numValid_Neighbors
    def getwheatInTank(self):
        return self.wheatInTank


class MyModel(ap.Model):
    def setup(self):
        # Crear un grid para el Harvester
        grid_size_harvester = (10, 10)
        self.grid_harvester = ap.Grid(self, shape=grid_size_harvester, track_empty=True)

        # Crear un grid para los Terrain
        grid_size_terrain = (10, 10)
        self.grid_terrain = ap.Grid(self, shape=grid_size_terrain)

        self.terrain_agents = ap.AgentList(self, 100, Terrain)
        self.grid_terrain.add_agents(self.terrain_agents,random=False)


        # Crear un Harvester en la posición inicial en el grid de Harvester
        self.harvester_agent = ap.AgentList(self, 1, Harvester)
        self.grid_harvester.add_agents(self.harvester_agent, [(0, 0)])

    def update(self):

        self.trimWheat = self.terrain_agents.select(self.terrain_agents.wheat == False)
        self.obstacleTer = self.terrain_agents.select(self.terrain_agents.obstacle == True)
        #self.possibleStep = self.terrain_agents.select(self.terrain_agents.valid_neighbors)
        # Stop simulation if all are trim
        #harvester_agent = self.grid_harvester.agents[0]
        self.harvester_agent.ObtainValid_neighbors()
        #print(self.harvester_agent.getnumValid_Neighbors())

        if self.harvester_agent.getnumValid_Neighbors()[0] == 0:
            self.stop()

    def step(self):
        # Lógica de simulación en cada paso

        # Obtener el único Harvester en el grid y llamar a moveToNoObstacle

        self.harvester_agent.moveToNoObstacle()

    def end(self):
      #harvester_agent = self.grid_harvester.agents[0]
      self.report('trim', self.harvester_agent.getwheatInTank())
      #print(self.harvester_agent.getwheatInTank())


# Crear una instancia del modelo y ejecutar la simulación
model = MyModel()
model.run()


NameError: ignored

In [ ]:

def animation_plot(model, ax):
    wheat_grid = model.grid_terrain.attr_grid('wheat')
    ap.gridplot(wheat_grid, cmap='viridis', ax=ax)
    ax.set_title(f"Time-step: {model.t}")

fig, ax = plt.subplots()
model = MyModel()
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())



